In [50]:
import gspread
from google.oauth2.service_account import Credentials
import json
import os

In [ ]:
def test_google_sheets_connection_fixed(sheet_url, path):
    """Исправленный тест подключения без создания файлов"""
    
    if not os.path.exists(path):
        print("❌ Файл credentials.json не найден!")
        return False
    
    try:
        # Загружаем credentials
        with open(path, 'r') as f:
            creds_data = json.load(f)
        
        print("✅ Файл credentials.json найден")
        print(f"📧 Email сервисного аккаунта: {creds_data['client_email']}")
        
        SCOPES = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive.readonly'
        ]
        
        # Создаем credentials
        creds = Credentials.from_service_account_file(path, scopes=SCOPES)
        gc = gspread.authorize(creds)
        
        print("✅ Аутентификация успешна")
        
        if sheet_url:
            # Тест 1: Открытие существующей таблицы
            print("\n🧪 Тест 1: Открытие существующей таблицы...")
            try:
                sheet = gc.open_by_url(sheet_url)
                worksheet = sheet.get_worksheet(0)  # Первый лист
                data = worksheet.get_all_values()
                print(f"✅ Таблица открыта: {sheet.title}")
                print(f"✅ Размер данных: {len(data)} строк, {len(data[0]) if data else 0} столбцов")
            except Exception as e:
                print(f"❌ Ошибка открытия таблицы: {e}")
                return False

        if sheet_url:
            print("\n🧪 Тест 3: Операция записи...")
            try:
                worksheet = sheet.get_worksheet(0)
                # Пробуем записать данные в маленький диапазон
                test_range = 'B2:C3'
                test_data = ['Data1', 'Data2']
                worksheet.append_row(test_data)
                print("✅ Запись данных: УСПЕХ")
                
                # Читаем обратно для проверки
                result = worksheet.get(test_range)
                print(f"✅ Проверка данных: {result}")
                
            except Exception as e:
                print(f"❌ Ошибка записи: {e}")
        
        print("\n🎉 Все тесты пройдены! Настройка корректна.")
        return True
        
    except Exception as e:
        print(f"❌ Критическая ошибка: {e}")
        return False

In [54]:
test_google_sheets_connection_fixed("https://docs.google.com/spreadsheets/d/1qVdDsfiTCZKMDyQsFrWhE4tmJVCuIoZ0CIWw05UCaRw/edit?gid=0#gid=0", "private/credentials.json")

✅ Файл credentials.json найден
📧 Email сервисного аккаунта: table-bot@table-pars.iam.gserviceaccount.com
✅ Аутентификация успешна

🧪 Тест 1: Открытие существующей таблицы...
✅ Таблица открыта: test
✅ Размер данных: 192 строк, 13 столбцов

🧪 Тест 3: Операция записи...
✅ Запись данных: УСПЕХ
✅ Проверка данных: [['Data2'], ['Лушникова Елизавета']]

🎉 Все тесты пройдены! Настройка корректна.


True